# Church image search demo

This is a demo that shows how to use multi-modal embeddings to perform a reverse image search

In [ ]:
#OpenSearch endpoint
#Host name should be WITHOUT https://

region = 'us-east-1' 
aos_host = "XXXXX.us-east-1.aoss.amazonaws.com"

In [ ]:
!pip install opensearch-py --quiet 

In [ ]:
# Standard library imports
import os
import re
import sys
import json
import base64
from io import BytesIO

# Other library imports
import boto3
import numpy as np
import seaborn as sns
from PIL import Image
from scipy.spatial.distance import cdist

In [ ]:
# Init Bedrock Runtime client
bedrock_client = boto3.client("bedrock-runtime")

In [ ]:
def titan_multimodal_embedding(
    image_path=None,  # maximum 2048 x 2048 pixels
    description=None, # English only and max input tokens 128
    dimension=1024,   # 1024 (default), 384, 256
    model_id="amazon.titan-embed-image-v1"
):
    payload_body = {}
    embedding_config = {
        "embeddingConfig": { 
             "outputEmbeddingLength": dimension
         }
    }

    # You can specify either text or image or both
    if image_path:
        with open(image_path, "rb") as image_file:
            input_image = base64.b64encode(image_file.read()).decode('utf8')
        payload_body["inputImage"] = input_image
    if description:
        payload_body["inputText"] = description

    assert payload_body, "please provide either an image and/or a text description"
    #print("\n".join(payload_body.keys()))

    response = bedrock_client.invoke_model(
        body=json.dumps({**payload_body, **embedding_config}), 
        modelId=model_id,
        accept="application/json", 
        contentType="application/json"
    )

    return json.loads(response.get("body").read())

Connect to OpenSearch and deploy the index to the collection

In [ ]:
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth
import boto3
import json
from botocore.config import Config

credentials = boto3.Session().get_credentials()
auth = AWSV4SignerAuth(credentials, region, service="aoss")

config = Config(
    region_name = region
)

aos_client = OpenSearch(
    hosts = [{'host': aos_host, 'port': 443}],
    http_auth = auth,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection
)

In [ ]:
image_index = {
    "aliases": {},
    "settings": {
      "index.knn": True,
      "analysis": {
          "analyzer": {
              "default": {
                  "type": "standard",
                  "stopwords": "_english_"
              }
          }
      }
    },
    "mappings": {
      "properties": {
        "title": {
            "type": "text",
            "store": True
        },
        "image_name": {
          "type": "text",
          "store": True
        },
        "image_description": {
          "type": "text",
          "store": True
        },
        "image_vector": {
          "type": "knn_vector",
          "dimension": 1024,
          "method": {
            "engine": "faiss",
            "name": "hnsw",
            "parameters": {
                "ef_construction": 128,
                "m": 24
            }
          }
        }
      }
    }
  }

In [ ]:
aos_client.indices.create(index="image_index",body=image_index,ignore=400)

Wait about 30-60 seconds before proceeding to the next step. OpenSearch needs time to make the index available.

In [ ]:
#Load the ./images/images.json file 
images_file = open('./images/images.json').read()
images = json.loads(images_file)
for item in images["images"]:
    name = item["image_name"]
    description = item["description"]

    embedding = titan_multimodal_embedding("./images/kb_images/" + name, description)
    
    #Load the data into OpenSearch
    aos_client.index(index="image_index", body={"image_name": name, "image_description": description, "image_vector": embedding['embedding']})    
    print(name)

Now display the results

In [ ]:
def get_thumbnail(path):
    i = Image.open(path)
    i.thumbnail((150, 150), Image.LANCZOS)
    return i

def image_base64(im):
    if isinstance(im, str):
        im = get_thumbnail(im)
    with BytesIO() as buffer:
        im.save(buffer, 'jpeg')
        return base64.b64encode(buffer.getvalue()).decode()

def image_formatter(im):
    return f'<img src="data:image/jpeg;base64,{image_base64(im)}">'

Wait about 30-60 seconds before moving on so OpenSearch as a chance to index the content

In [ ]:
import pandas as pd
from IPython.display import HTML

results = titan_multimodal_embedding(image_path="./images/bern_switzerland_temple_lds.jpeg")
#results = titan_multimodal_embedding(image_path="./images/young_woman_prayer_table.jpeg")
#results = titan_multimodal_embedding(image_path="./images/bloch_sample.jpg")
#results = titan_multimodal_embedding(image_path="./images/trek_west.jpeg") 
#results = titan_multimodal_embedding(description="Dogs in wigs")

#results = titan_multimodal_embedding(image_path="./images/christ_ordaining_the_apostles.jpeg")
#results = titan_multimodal_embedding(description="Jesus cleansing the temple")
#results = titan_multimodal_embedding(image_path="./images/salt_lake_temple.jpeg")

query={
    "query": {
        "knn": {
            "image_vector":{
                "vector":results['embedding'],
                "min_score": 0.53
            }
        }
    }
}

res = aos_client.search(index="image_index", 
                       body=query,
                       stored_fields=["image_name","image_description"])
print("Got %d Hits:" % res['hits']['total']['value'])

query_result=[]
for hit in res['hits']['hits']:
    row=[hit['_score'],hit['fields']['image_name'][0],hit['fields']['image_description'][0]]
    query_result.append(row)

df = pd.DataFrame(query_result, columns=['score','image_name','image_description'])
df['image'] = df.image_name.map(lambda f: get_thumbnail("./images/kb_images/" + f))
HTML(df[['score', 'image_name', 'image_description', 'image']].to_html(formatters={'image': image_formatter}, escape=False))